In [2]:
import os
import random
import math
import cv2
import json
import numpy as np
import pandas as pd

In [36]:
import numpy as np

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd
from shutil import copyfile,copy2

In [6]:
from keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'keras'

In [5]:
pip show keras

Name: keras
Version: 2.7.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: c:\users\ruhiy\.conda\envs\distracted_driver\lib\site-packages
Requires: 
Required-by: tensorflow
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip show numpy

Name: numpy
Version: 1.21.2
Summary: NumPy is the fundamental package for array computing with Python.
Home-page: https://www.numpy.org
Author: Travis E. Oliphant et al.
Author-email: 
License: BSD
Location: c:\users\ruhiy\.conda\envs\distracted_driver\lib\site-packages
Requires: 
Required-by: tensorflow, tensorboard, scipy, pandas, opt-einsum, numexpr, mkl-random, mkl-fft, Keras-Preprocessing, h5py, Bottleneck
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip show tensorflow-gpu

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip show pandas

Name: pandas
Version: 1.3.4
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: c:\users\ruhiy\.conda\envs\distracted_driver\lib\site-packages
Requires: python-dateutil, pytz, numpy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip show IPython

Name: ipython
Version: 7.29.0
Summary: IPython: Productive Interactive Computing
Home-page: https://ipython.org
Author: The IPython Development Team
Author-email: ipython-dev@python.org
License: BSD
Location: c:\users\ruhiy\.conda\envs\distracted_driver\lib\site-packages
Requires: matplotlib-inline, prompt-toolkit, backcall, traitlets, decorator, jedi, pygments, colorama, pickleshare, setuptools
Required-by: ipykernel
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
tf.__version__

'2.7.0'

In [2]:
import keras
keras.__version__

'2.7.0'

In [5]:
BASE_DIR = r'C:\Users\ruhiy\Documents\Machine Learning\Distracted Driver Detection'

In [7]:
DATA_DIR = os.path.join(BASE_DIR, r'state-farm-distracted-driver-detection')
print(DATA_DIR)

C:\Users\ruhiy\Documents\Machine Learning\Distracted Driver Detection\state-farm-distracted-driver-detection


In [8]:
TRAIN_DIR = os.path.join(DATA_DIR, r'imgs\train')
TEST_DIR = os.path.join(DATA_DIR, r'imgs\test')

Create training, test, and validation datasets

In [9]:
csv = pd.read_csv(os.path.join(DATA_DIR, 'driver_imgs_list.csv'))

In [10]:
csv

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg
...,...,...,...
22419,p081,c9,img_56936.jpg
22420,p081,c9,img_46218.jpg
22421,p081,c9,img_25946.jpg
22422,p081,c9,img_67850.jpg


In [16]:
csv = csv.values

In [17]:
csv

array([['p002', 'c0', 'img_44733.jpg'],
       ['p002', 'c0', 'img_72999.jpg'],
       ['p002', 'c0', 'img_25094.jpg'],
       ...,
       ['p081', 'c9', 'img_25946.jpg'],
       ['p081', 'c9', 'img_67850.jpg'],
       ['p081', 'c9', 'img_9684.jpg']], dtype=object)

In [18]:
csv[0][1]

'c0'

In [35]:
# create directory structure for storing datasets
datasets = ['train', 'test', 'validate']
for i in datasets: 
    dataset_path = os.path.join(BASE_DIR, i)
    if (not(os.path.exists(dataset_path))):
        os.mkdir(dataset_path)
    for j in range(0, 10): 
        class_path = os.path.join(dataset_path, 'c' + str(j))
        if (not(os.path.exists(class_path))):
            os.mkdir(class_path)

In [39]:
# copy images from original train dataset into folders for train, 
# test, and validate while maintaining original folder structure.
# divide the images in the train dataset as follows:
# 25% of train images are test images
# 10% of the remaining 75% are validation images
# 90% of the remaining 75% are used for training

i = j = 1
for row in csv:
    classname = row[1]
    img = row[2]
    if i % 4 == 0: # copy image into test dataset
        dest_img = os.path.join(BASE_DIR, datasets[1], classname, img)
        src_img = os.path.join(TRAIN_DIR, classname, img) 
        copy2(src_img, dest_img)
    else:
        if j % 10 == 0: # copy image into validate dataset
            dest_img = os.path.join(BASE_DIR, datasets[2], classname, img)
            src_img = os.path.join(TRAIN_DIR, classname, img) 
            copy2(src_img, dest_img)
        else:
            dest_img = os.path.join(BASE_DIR, datasets[0], classname, img)
            src_img = os.path.join(TRAIN_DIR, classname, img) 
            copy2(src_img, dest_img)
    
    i += 1
    j += 1
    

In [61]:
# check how many images are copied to all directories
from functools import reduce
def count_files(dir):
    return reduce(lambda x, y: x + y, ([len(files) for r, d, files in os.walk(dir)]))

sum = 0
for i in datasets:
    sum += count_files(os.path.join(BASE_DIR, i))
    print(sum)

15697
21303
22424


In [62]:
def preProcess(x):
    mean_pixel = [103.939, 116.779, 123.68]
    x[0, :, :] -= mean_pixel[0]
    x[1, :, :] -= mean_pixel[1]
    x[2, :, :] -= mean_pixel[2]
    return x

In [63]:
# this is the augmentation configuration we will use for training

train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=False,
        fill_mode='nearest',
        preprocessing_function = preProcess)

# # this is the augmentation configuration we will use for testing:
# # only rescaling
# train_data_dir = '../Labelled Train Data/'
test_datagen = ImageDataGenerator(preprocessing_function = preProcess)

train_data_dir = os.path.join(BASE_DIR, datasets[0])
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical')


validate_data_dir = os.path.join(BASE_DIR, datasets[2])
validation_generator = test_datagen.flow_from_directory(
        validate_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='categorical')

NameError: name 'ImageDataGenerator' is not defined